# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,shukat as sufi,31.2600,34.2826,25.52,57,4,2.76,PS,1698718268
1,1,waitangi,-43.9535,-176.5597,15.61,79,100,6.71,NZ,1698718131
2,2,strezhevoy,60.7333,77.5889,-13.20,96,12,2.24,RU,1698718269
3,3,bull savanna,17.8868,-77.5902,24.54,85,64,1.89,JM,1698718269
4,4,kegeyli shahar,42.7767,59.6078,5.20,74,0,3.80,UZ,1698718270


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:

# Configure the map plot
city_data_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"

)

# Display the map
city_data_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
    ]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna(how = "any", axis = 0)

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
86,86,tazacorte,28.6290,-17.9293,22.69,93,0,2.24,ES,1698718318
93,93,shingu,33.7333,135.9833,21.94,44,0,1.31,JP,1698718319
153,153,penalva,-3.2942,-45.1736,25.11,80,0,1.62,BR,1698718343
159,159,jirja,26.3383,31.8916,24.86,50,0,2.45,EG,1698718345
192,192,iranshahr,27.2025,60.6848,22.34,40,0,2.06,IR,1698718359
194,194,saint-pierre,-21.3393,55.4781,22.97,64,0,3.60,RE,1698718180
256,256,carnarvon,-24.8667,113.6333,25.94,45,0,3.58,AU,1698718385
274,274,maceio,-9.6658,-35.7353,22.84,94,0,1.54,BR,1698718394
287,287,gua,22.2000,85.3833,21.52,68,0,0.77,IN,1698718398
335,335,mirpur sakro,24.5474,67.6277,26.26,48,0,3.34,PK,1698718417


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.read_csv("output_data/cities.csv")[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,shukat as sufi,PS,31.2600,34.2826,57,
1,waitangi,NZ,-43.9535,-176.5597,79,
2,strezhevoy,RU,60.7333,77.5889,96,
3,bull savanna,JM,17.8868,-77.5902,85,
4,kegeyli shahar,UZ,42.7767,59.6078,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel

radius = 10000

params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filters"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # # Make an API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # # Convert the API response to JSON format
    name_address = response.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
shukat as sufi - nearest hotel: مول العرب(رفح)
waitangi - nearest hotel: Hotel Chathams
strezhevoy - nearest hotel: Гостиница "Сибирячка"
bull savanna - nearest hotel: Junction Guest House
kegeyli shahar - nearest hotel: Citizen Hotel
uturoa - nearest hotel: Hawaiki Nui hotel
mount isa - nearest hotel: Ibis Styles
adamstown - nearest hotel: Chez Bianca & Benoit
puerto natales - nearest hotel: Dorotea Patagonia Hostel
bredasdorp - nearest hotel: Victoria Hotel
west island - nearest hotel: Cocos Village Bungalows
blackmans bay - nearest hotel: Villa Howden
bilibino - nearest hotel: No hotel found
castillos - nearest hotel: A mi gente
zacatecas - nearest hotel: HOTEL Plata
port-aux-francais - nearest hotel: Keravel
bethel - nearest hotel: Hampton Inn Danbury
hermanus - nearest hotel: Aloe guest house
howrah - nearest hotel: Sun India Guest House
palu - nearest hotel: Ujung Pandang Hotel
albany - nearest hotel: Tru
talnakh - nearest hotel: Талнах
constantia - nearest 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,shukat as sufi,PS,31.2600,34.2826,57,مول العرب(رفح)
1,waitangi,NZ,-43.9535,-176.5597,79,Hotel Chathams
2,strezhevoy,RU,60.7333,77.5889,96,"Гостиница ""Сибирячка"""
3,bull savanna,JM,17.8868,-77.5902,85,Junction Guest House
4,kegeyli shahar,UZ,42.7767,59.6078,74,Citizen Hotel
...,...,...,...,...,...,...
538,codrington,AU,-38.2667,141.9667,51,54 on Bank
539,akwanga,NG,8.9167,8.3833,81,Jempo Hotel
540,la passe,FR,45.5549,-0.8967,84,L'Estuaire
541,beichengqu,CN,40.4394,113.1536,35,春江大酒店


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 0.01,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]

   
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)